1. 特征工程的目的是从原始数据中提取特征以供算法和模型使用。sklearn库包含众多函数

In [2]:
#load data from sklearn 
from sklearn.datasets import load_iris
iris = load_iris()
#iris数据有特征矩阵iris.data，尺寸为【150，4】，标签iris.target, iris.feature_names和iris.target_names

2. 数据预处理
 2.1 无量纲化，包括标准化（前提为正太分布，标准化后为标准正态分布）；区间缩放，比如除以最值，区间变为【0,1】或【-1，1】
 注：如果对输出结果范围有要求，用区间缩放；如果数据较为稳定，不存在极端的最大最小值，用区间缩放；如果数据存在异常值和较多噪音，用标 准化，可以间接通过中心化避免异常值和极端值的影响

In [13]:
print('original iris No.1-3: \n', iris.data[0:3])
#标准化
from sklearn.preprocessing import StandardScaler
iris_standard=StandardScaler().fit_transform(iris.data[0:3])
print('after standardscaler iris No.1-3: \n', iris_standard)
#区间缩放
from sklearn.preprocessing import MinMaxScaler
iris_MinMax=MinMaxScaler().fit_transform(iris.data[0:3])
print('after maxscaler iris No.1-3: \n', iris_MinMax)


original iris No.1-3: 
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
after standardscaler iris No.1-3: 
 [[ 1.22474487e+00  1.29777137e+00  7.07106781e-01 -1.00000000e+00]
 [ 5.43895982e-15 -1.13554995e+00  7.07106781e-01 -1.00000000e+00]
 [-1.22474487e+00 -1.62221421e-01 -1.41421356e+00 -1.00000000e+00]]
after maxscaler iris No.1-3: 
 [[1.  1.  1.  0. ]
 [0.5 0.  1.  0. ]
 [0.  0.4 0.  0. ]]


此外，还有归一化，他是按照特征矩阵的行处理数据（注意标准化和区间缩放都是根据列向量，即不同样本的同一个特征变量）.归一化的目的是在点乘运算或是其他和函数计算相似性时，拥有统一的标准，也就是转换为“单位向量”。

In [15]:
from sklearn.preprocessing import Normalizer 
iris_norm = Normalizer().fit_transform(iris.data[0:3])
print('after normalizer iris No.1-3: \n', iris_norm)

after normalizer iris No.1-3: 
 [[0.80377277 0.55160877 0.22064351 0.0315205 ]
 [0.82813287 0.50702013 0.23660939 0.03380134]
 [0.80533308 0.54831188 0.2227517  0.03426949]]


2.2 对定量特征二值化

In [16]:
from sklearn.preprocessing import Binarizer
iris_Binary=Binarizer(threshold=3).fit_transform(iris.data[:3])
print('after binarizer iris No.1-3: \n', iris_Binary)

after binarizer iris No.1-3: 
 [[1. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 0. 0.]]


2.3 One-hot编码，把定性变量转换为one-hot的哑编码 eg. (1,2,3)->[【1,0,0】【0,1,0】【0,0,1】]


In [18]:
from sklearn.preprocessing import OneHotEncoder
iris_target_OneHot= OneHotEncoder().fit_transform(iris.target.reshape(-1,1))
print(iris_target_OneHot.shape)

(150, 3)


2.4 缺失值计算。 使用Imputer计算缺失值，参数missing_value为缺失值的表示形式（默认为NaN),strategy为填充方式，默认为均值mean


In [21]:
import numpy as np
from sklearn.preprocessing import Imputer
iris151 = np.vstack((np.array([np.nan,np.nan,np.nan,np.nan]),iris.data))
print(iris151[0],iris151.shape,iris.data.shape)
iris151new = Imputer(missing_values="NaN", strategy="mean", axis=0, verbose=0, copy=True).fit_transform(iris151)
print(iris151new[0])

[nan nan nan nan] (151, 4) (150, 4)
[5.84333333 3.05733333 3.758      1.19933333]
/home/chumiao/.conda/envs/pytorch/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


2.5 数据转换，包括多项式转换，任意自定义函数log,exp转换
 

In [29]:
from sklearn.preprocessing import PolynomialFeatures,FunctionTransformer
#多项式转换，利用现有的变量相乘增加为指定阶数的新变量
iris_Poly2=PolynomialFeatures(degree=2).fit_transform(iris.data[:3])
iris_Poly3=PolynomialFeatures(degree=3).fit_transform(iris.data[:3])
print(iris.data[:3].shape)
print(iris_Poly2.shape)
print(iris_Poly3.shape)
#自定义转换函数
iris_log = FunctionTransformer(func=np.log1p).fit_transform(iris.data[:3])
print(iris_log.shape)

(3, 4)
(3, 15)
(3, 35)
(3, 4)


3. 特征选择。
在预处理之后，对有意义的特征进行筛选，主要考察特征的①发散性（如果不发散var=0,该特征无意义）；②与目标的相关性
方法为3中：① Filter (根据相关性和发散性按照阈值来选择)；② Wrapper (递归特征消除法) ③ Embedded (使用机器学习算法和模型，按照特征的权值系数选择特征，高级的Filter)

3.1 Filter

In [34]:
#方差选择法
from sklearn.feature_selection import VarianceThreshold
iris_Var_Select = VarianceThreshold(threshold=3).fit_transform(iris.data)
print(iris_Var_Select.shape)
iris_Var_Select = VarianceThreshold(threshold=0.5).fit_transform(iris.data)
print(iris_Var_Select.shape)

(150, 1)
(150, 3)


In [ ]:
# 单变量分析法 univariate feature selection,基于在统计中特征与输出的相关系数、回归系数、互信息等进行筛选
# - 其中筛选函数有按保留个数筛选SelectKBest,按百分比筛选SelectPercentile，按关注的统计特性(false positive rate, false discovery rate )筛选SelectFpr,SelctFdr
# - 统计特征：For regression: f_regression, mutual_info_regression; For classification: chi2, f_classif, mutual_info_classif

In [74]:

from sklearn.feature_selection import SelectKBest,chi2
from scipy.stats import pearsonr
a = lambda X, Y: np.abs((np.array(list(map(lambda x:pearsonr(x, Y), X.T))).T)[0]) 
#a为lambda函数，计算特征矩阵m个参数X.T与Y的相关性，得到二维矩阵(m,2),其中第一维代表相关系数，第二维代表显著性p；SelectKBest只能是一维度的输出结果，所有取了[0]
iris_Person = SelectKBest(a, k=2).fit(iris.data, iris.target) #这里先使用了fit再调用get_support,从而得到保留下来的feature索引
print(iris_Person.get_support(indices=True))
iris_Person_new = SelectKBest(a, k=2).fit_transform(iris.data, iris.target) #fit_transform返回特征过滤后保留下的特征数据集。
print(iris_Person_new.shape)
iris_chi2 = SelectKBest(chi2, k=2).fit(iris.data, iris.target) #经典的卡方检验是检验定性自变量对定性因变量的相关性
print(iris_chi2.get_support(indices=True))



[2 3]
(150, 2)
[2 3]


In [60]:
#互信息法
from minepy import MINE
def mic(x,y):
    m = MINE()
    m.compute_score(x,y)
    return m.mic()
a = lambda X,Y: np.array(list(map(lambda x: mic(x,Y), X.T))).T
iris_MultInfo = SelectKBest(a,k=2).fit_transform(iris.data,iris.target)
print(iris_MultInfo[:3])


[[1.4 0.2]
 [1.4 0.2]
 [1.3 0.2]]


3.2 Wrapper
递归消除特征法使用一个基模型(自己可以选择）来进行多轮训练，每轮训练后，消除若干权值系数的特征，再基于新的特征集进行下一轮训练，直到满足剩余的特征个数。使用feature_selection库的RFE类/RFECV类（带cross-validation)来选择特征的代码

In [70]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel='linear')
selector = RFE(estimator,n_features_to_select=2,step=1)
selector = selector.fit(iris.data,iris.target)
print(selector.support_)
print(selector.ranking_)
iris_selector_data = selector.transform(iris.data)
print(iris_selector_data.shape)


[False False  True  True]
[2 3 1 1]
(150, 2)


In [79]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC #svc classification ,svr regression
from sklearn.model_selection import StratifiedKFold
estimator = SVC(kernel='linear')
selector = RFECV(estimator,step=1,cv=StratifiedKFold(2),scoring='accuracy',min_features_to_select=1)
selector = selector.fit(iris.data,iris.target)
print("optimal num of features",selector.n_features_) #自动计算optimal的feature num，不需要像REF一样指定

(150, 4) (150,)
optimal num of features 4


3.3 SelectFromModel 像是Variance_Threshold 和 RFE的一个综合。利用任何estimator来对每个特征进行评分，然后可以根据threshold来选择保留的特征；除了设置阈值外，还可以设定max_features.

In [93]:
# L1和L2惩罚的estimator:In particular, sparse estimators useful for this purpose are the Lasso for regression, and of LogisticRegression and LinearSVC for classification
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
estimator_l1 = LogisticRegression(penalty='l1',C=0.01)
estimator_l2 = LogisticRegression(penalty='l2',C=0.01)
Selector_l1 = SelectFromModel(estimator_l1).fit(iris.data,iris.target)
print(Selector_l1.estimator_.coef_)
print(Selector_l1.threshold_, Selector_l1.get_support())

Selector_l2 = SelectFromModel(estimator_l2).fit(iris.data,iris.target)
print(Selector_l2.estimator_.coef_)
print(Selector_l2.threshold_, Selector_l2.get_support())

[[ 0.          0.         -0.18016819  0.        ]
 [-0.03183986  0.          0.          0.        ]
 [-0.00677759  0.          0.          0.        ]]
1e-05 [ True False  True False]
[[ 0.03366372  0.24569333 -0.46437192 -0.20710127]
 [-0.08039609 -0.17543666  0.09803087  0.00640839]
 [-0.18404126 -0.22424632  0.29110621  0.18614789]]
0.5491609826547749 [False  True  True False]


In [99]:
# Tree-based estimators: sklearn.tree or sklearn.ensemble 
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=50)
# print(clf.feature_importances_)
selector = SelectFromModel(clf).fit(iris.data,iris.target)
print(selector.threshold_,selector.get_support())

0.25 [False False  True  True]


4. 降维。 在特征选择过后，保留下来的特征仍然很多，矩阵运算过大训练时间长。常见的降维包括主成分分析法（PCA）和线性判别分析（LDA）：线性判别分析本身也是一个分类模型。PCA和LDA有很多的相似点，其本质是要将原始的样本映射到维度更低的样本空间中，但是PCA和LDA的映射目标不一样：PCA是为了让映射后的样本具有最大的发散性；而LDA是为了让映射后的样本有最好的分类性能。所以说PCA是一种无监督的降维方法，而LDA是一种有监督的降维方法。

In [100]:
from sklearn.decomposition import PCA
data_PCA = PCA(n_components=2).fit_transform(iris.data)
print(iris.data[:3])
print(data_PCA[:3])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]]
[[-2.68412563  0.31939725]
 [-2.71414169 -0.17700123]
 [-2.88899057 -0.14494943]]


In [105]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
data_LDA = LDA(n_components=2).fit_transform(iris.data,iris.target)
print(data_LDA[:3])

[[ 8.06179978  0.30042062]
 [ 7.12868772 -0.78666043]
 [ 7.48982797 -0.26538449]]
